In [3]:
import numpy as np

In [4]:
NUM_KNAPSACKS = 3
NUM_ITEMS = 10 
NUM_DIMENSIONS = 2 

In [5]:
VALUES = np.random.randint(0, 100, size=NUM_ITEMS)
WEIGHTS = np.random.randint(0, 100, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = np.random.randint(
    0, 100 * NUM_ITEMS // NUM_KNAPSACKS, size=(NUM_KNAPSACKS, NUM_DIMENSIONS)
)

In [6]:
CONSTRAINTS

array([[176, 193],
       [297,  20],
       [ 14, 125]])

In [7]:
import numpy as np
from icecream import ic
from tqdm import trange  

def cost(solution):
    # checks each column (each item) across all knapsacks
    # returns a boolean vector of length NUM_ITEMS, where each entry is True if that item appears in at least one knapsack
    return VALUES[np.any(solution, axis=0)].sum()

def is_feasible(total_weights):
    return not np.any(total_weights > CONSTRAINTS)

def random_solution():
    # in this approach we are maintaining incremental weights 
    sol = np.zeros((NUM_KNAPSACKS, NUM_ITEMS), dtype=bool)
    total_weights = np.zeros((NUM_KNAPSACKS, NUM_DIMENSIONS))
    items = np.random.permutation(NUM_ITEMS)
    for i in items:
        k = np.random.randint(NUM_KNAPSACKS)
        if np.all(total_weights[k] + WEIGHTS[i] <= CONSTRAINTS):
            sol[k, i] = True # add item to solution
            total_weights[k] += WEIGHTS[i]
    return sol, total_weights

# old_knapsack to remove from, new_knapsack to assign an item
def update_weights(total_weights, item_idx, old_k, new_k):
    if old_k is not None:
        total_weights[old_k] -= WEIGHTS[item_idx]
    if new_k is not None:
        total_weights[new_k] += WEIGHTS[item_idx]

def get_neighbors(sample_size=20):
    items = np.random.choice(NUM_ITEMS, size=sample_size, replace=False) # distinct random items
    knapsacks = np.random.randint(0, NUM_KNAPSACKS, size=sample_size)
    return list(zip(items, knapsacks))

def hill_climbing(max_iter=300, neighbor_sample=200):
    sol, total_weights = random_solution()
    best_val = cost(sol)
    best_sol = sol.copy()
    best_weights = total_weights.copy()

    for it in trange(max_iter, desc="Hill Climbing"):
        neighbors = get_neighbors(sample_size=neighbor_sample)
        for i, k in neighbors:
            # find current knapsack of item i (if any)
            old_k = np.where(sol[:, i])[0]
            old_k = old_k[0] if len(old_k) > 0 else None
            # skip if theres is no actual change
            if old_k == k:
                continue
            # else try moving from old_knapsack to k
            update_weights(total_weights, i, old_k, k)
            feasible = is_feasible(total_weights)
            if feasible:
                # apply move
                if old_k is not None:
                    sol[old_k, i] = False
                sol[k, i] = True
                new_val = cost(sol)
                if new_val > best_val:
                    best_val = new_val
                    best_sol = sol.copy()
                    best_weights = total_weights.copy()
                else: # rollback
                    if old_k is not None:
                        sol[old_k, i] = True
                    sol[k, i] = False
                    update_weights(total_weights, i, k, old_k)
            else:
                # rollback weights if infeasible
                update_weights(total_weights, i, k, old_k)

    return best_sol, best_val, best_weights

## TEST PROBLEMS

In [22]:
# Problem 1:
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 3
NUM_ITEMS = 20
NUM_DIMENSIONS = 2
VALUES = rng.integers(0, 100, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 100, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(
    0, 100 * NUM_ITEMS // NUM_KNAPSACKS, size=(NUM_KNAPSACKS, NUM_DIMENSIONS)
)

# ---------- Run ---------- #
best_solution, best_value, best_weights = hill_climbing(neighbor_sample=10)
ic(best_value)
ic(best_solution.astype(int))

Hill Climbing: 100%|██████████| 300/300 [00:00<00:00, 5902.07it/s]
ic| best_value: 1000
ic| best_solution.astype(int): array([[1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1],
                                      [0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0],
                                      [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0]])


array([[1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0]])

In [27]:
# Problem 2:
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 10
NUM_ITEMS = 100
NUM_DIMENSIONS = 10
VALUES = rng.integers(0, 1000, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 1000, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(
    1000 * 2, 1000 * NUM_ITEMS // NUM_KNAPSACKS, size=(NUM_KNAPSACKS, NUM_DIMENSIONS)
)

# ---------- Run ---------- #
best_solution, best_value, best_weights = hill_climbing(neighbor_sample=50)
ic(best_value)
# ic(best_solution.astype(int))

Hill Climbing: 100%|██████████| 300/300 [00:00<00:00, 1843.74it/s]
ic| best_value: 31989


31989

In [24]:
# Problem 3:
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 100
NUM_ITEMS = 5000
NUM_DIMENSIONS = 100
VALUES = rng.integers(0, 1000, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 1000, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(
    1000 * 10, 1000 * 2 * NUM_ITEMS // NUM_KNAPSACKS, size=(NUM_KNAPSACKS, NUM_DIMENSIONS)
)

# ---------- Run ---------- #
best_solution, best_value, best_weights = hill_climbing(neighbor_sample=1000)
ic(best_value)
# ic(best_solution.astype(int))

Hill Climbing: 100%|██████████| 300/300 [00:05<00:00, 56.40it/s]
ic| best_value: 1180091


1180091